In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import geoplot
import pickle
import geoplot.crs as gcrs
import statsmodels.api as sm

/home/jasonllu/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.9.0-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
node_degree_df = pd.read_pickle('../../data_process/node_degree_df.pickle')
edge_degree_df = pd.read_pickle('../../data_process/edge_degree_df.pickle')

In [3]:
len(node_degree_df["SA2_MAIN16"].values)

103

In [4]:
edge_degree_df.columns

Index(['timestamp', 'destination', 'state', 'origin', 'unique_agents',
       'sum_stay_duration', 'total_stays', 'OD', 'class_ART', 'class_BUS',
       'class_COLL', 'class_FREE', 'class_HWY', 'class_LOCL', 'class_SUBA',
       'class_TRK2', 'class_TRK4', 'class_UND', 'num_jobs_000_persons_origin',
       'sa2_code16_x', 'median_income_per_job_aud_origin',
       'num_jobs_000_persons_destination', 'sa2_code16_y',
       'median_income_per_job_aud_destination', 'od_duration_value',
       'od_distance_value', 'road_counts', 'num_nodes_x', 'num_1degree_x',
       'num_2degree_x', 'num_3degree_x', 'num_4degree_x',
       'num_greater5degree_x', 'num_nodes_y', 'num_1degree_y', 'num_2degree_y',
       'num_3degree_y', 'num_4degree_y', 'num_greater5degree_y'],
      dtype='object')

In [5]:
mount_path = "/mnt/c/Users/jason/Dropbox (MIT)/"
#mount_path = "/Users/shenhaowang/Dropbox (MIT)/project_media_lab_South_Australia/"

In [6]:
poi_df = gpd.read_file(mount_path + "/SA data/dataSA/points/points.shp")

In [7]:
poi_df.head()

,osm_id,timestamp,name,type,geometry
0,29240333,None,None,traffic_signals,POINT (138.60119 -34.91082)
1,29398542,None,None,crossing,POINT (138.59423 -34.76718)
2,29491931,None,Adelaide Railway Station Tram Stop (To Adelaide,tram_stop,POINT (138.59739 -34.92171)
3,29491942,None,None,level_crossing,POINT (138.59949 -34.92299)
4,29492063,None,None,traffic_signals,POINT (138.59351 -34.92330)


In [8]:
sa2_south_au = gpd.read_file("../../data_process/shapefiles/sa2_south_au.shp")

In [9]:
sa2_south_au.shape

(172, 13)

In [10]:
sa2_south_au.head()

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
0,401011001,41001,Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,10.4824,"POLYGON ((138.58096 -34.93024, 138.58094 -34.9..."
1,401011002,41002,North Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,5.0909,"POLYGON ((138.58214 -34.90424, 138.58287 -34.9..."
2,401021003,41003,Adelaide Hills,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,364.4390,"POLYGON ((138.75610 -34.82107, 138.75567 -34.8..."
3,401021004,41004,Aldgate - Stirling,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,117.2141,"POLYGON ((138.69440 -34.98807, 138.69514 -34.9..."
4,401021005,41005,Hahndorf - Echunga,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,110.1516,"POLYGON ((138.78877 -35.02474, 138.78889 -35.0..."


In [11]:
print(len(poi_df.geometry))

19701


In [12]:
# #make sure all points only in one area
# not_found_count = 0
# for i, centroid in enumerate(poi_df.geometry):
#     found = False
#     count = 0
#     l = []
#     if i%100 == 0: print(i)
#     for i, row in sa2_south_au.iterrows():
#         if row["geometry"].contains(centroid):
#             count += 1
#             l.append(row["geometry"])
#             found = True
#     if not found:
#         print("not found")
#         not_found_count += 1
#     if len(l) > 1:
#         print(l)
#make sure all points only in one area
# not_found_count = 0
# for i, centroid in enumerate(poi_df.geometry):
#     found = False
#     count = 0
#     l = []
#     if i%100 == 0: print(i)
#     for i, row in sa2_south_au.iterrows():
#         if centroid.within(row["geometry"]):
#             count += 1
#             l.append(row["geometry"])
#             found = True
#     if not found:
#         print("not found")
#         not_found_count += 1
#     if len(l) > 1:
#         print(l)

In [13]:
# print(not_found_count)

In [14]:
# type(sa2_south_au["SA2_MAIN16"].values[0])

In [15]:
# sa_codes_poi = []
# for i, centroid in enumerate(poi_df.geometry):
#     if i%100 == 0: print(i)
#     found = False
#     for i, row in sa2_south_au.iterrows():
#         if row["geometry"].contains(centroid):
#             sa_codes_poi.append(row["SA2_MAIN16"])
#             found = True
#             break
#     if not found:
#         sa_codes_poi.append("0")
            

In [16]:
#poi_df.to_pickle("../../data_process/poi_df.pickle")
poi_df = pd.read_pickle("../../data_process/poi_df.pickle")
#poi_df["SA2_MAIN16"] = sa_codes_poi

In [17]:
print(len(poi_df))
poi_df = poi_df[poi_df["SA2_MAIN16"]!="0"]
print(len(poi_df))

19701
19679


In [18]:
poi_df.head()

,osm_id,timestamp,name,type,geometry,SA2_MAIN16
0,29240333,None,None,traffic_signals,POINT (138.60119 -34.91082),401011002
1,29398542,None,None,crossing,POINT (138.59423 -34.76718),402041044
2,29491931,None,Adelaide Railway Station Tram Stop (To Adelaide,tram_stop,POINT (138.59739 -34.92171),401011001
3,29491942,None,None,level_crossing,POINT (138.59949 -34.92299),401011001
4,29492063,None,None,traffic_signals,POINT (138.59351 -34.92330),401011001


In [19]:
count = poi_df.groupby(["SA2_MAIN16"],as_index=False).aggregate(["count"])

In [20]:
count.head()

,osm_id,timestamp,name,type,geometry
,count,count,count,count,count
SA2_MAIN16,,,,,
401011001,3288,0,757,3288,3288
401011002,756,0,152,756,756
401021004,279,0,181,279,279
401021010,153,0,56,153,153
401031011,276,0,141,276,276


In [21]:
count.shape

(76, 5)

In [22]:
count.columns

MultiIndex([(   'osm_id', 'count'),
            ('timestamp', 'count'),
            (     'name', 'count'),
            (     'type', 'count'),
            ( 'geometry', 'count')],
           )

In [23]:
split_count = poi_df.groupby(["SA2_MAIN16","type"],as_index=False).aggregate(["count"])

In [24]:
split_count

osm_id timestamp  name geometry
                            count     count count    count
SA2_MAIN16 type                                           
401011001  apartment            2         0     2        2
           arts_centre          3         0     3        3
           artwork              7         0     4        7
           atm                 11         0     2       11
           attraction          10         0    10       10
...                           ...       ...   ...      ...
404031109  toilets              1         0     0        1
           traffic_signals      4         0     0        4
           turning_circle      10         0     0       10
           vending_machine      1         0     0        1
           viewpoint            1         0     0        1

[1799 rows x 4 columns]

In [25]:
for i, row in split_count.iterrows():
    print(row)

osm_id     count    2
timestamp  count    0
name       count    2
geometry   count    2
Name: (401011001, apartment), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    3
geometry   count    3
Name: (401011001, arts_centre), dtype: int64
osm_id     count    7
timestamp  count    0
name       count    4
geometry   count    7
Name: (401011001, artwork), dtype: int64
osm_id     count    11
timestamp  count     0
name       count     2
geometry   count    11
Name: (401011001, atm), dtype: int64
osm_id     count    10
timestamp  count     0
name       count    10
geometry   count    10
Name: (401011001, attraction), dtype: int64
osm_id     count    24
timestamp  count     0
name       count    24
geometry   count    24
Name: (401011001, bank), dtype: int64
osm_id     count    23
timestamp  count     0
name       count    22
geometry   count    23
Name: (401011001, bar), dtype: int64
osm_id     count    32
timestamp  count     0
name       count     0
geometry   cou

Name: (401031011, public_bookcase), dtype: int64
osm_id     count    9
timestamp  count    0
name       count    9
geometry   count    9
Name: (401031011, restaurant), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    1
geometry   count    1
Name: (401031011, school), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    0
geometry   count    3
Name: (401031011, telephone), dtype: int64
osm_id     count    4
timestamp  count    0
name       count    0
geometry   count    4
Name: (401031011, toilets), dtype: int64
osm_id     count    9
timestamp  count    0
name       count    0
geometry   count    9
Name: (401031011, traffic_signals), dtype: int64
osm_id     count    46
timestamp  count     0
name       count     0
geometry   count    46
Name: (401031011, turning_circle), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    0
geometry   count    3
Name: (401031011, turning_loop), dtype: int64
osm_id     count    

Name: (401061021, bench), dtype: int64
osm_id     count    4
timestamp  count    0
name       count    0
geometry   count    4
Name: (401061021, bicycle_parking), dtype: int64
osm_id     count    67
timestamp  count     0
name       count    67
geometry   count    67
Name: (401061021, bus_stop), dtype: int64
osm_id     count    5
timestamp  count    0
name       count    5
geometry   count    5
Name: (401061021, cafe), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    1
geometry   count    1
Name: (401061021, childcare), dtype: int64
osm_id     count    67
timestamp  count     0
name       count     0
geometry   count    67
Name: (401061021, crossing), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    1
geometry   count    1
Name: (401061021, dentist), dtype: int64
osm_id     count    7
timestamp  count    0
name       count    2
geometry   count    7
Name: (401061021, drinking_water), dtype: int64
osm_id     count    2
timestamp  c

Name: (402041043, childcare), dtype: int64
osm_id     count    17
timestamp  count     0
name       count     0
geometry   count    17
Name: (402041043, crossing), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    0
geometry   count    2
Name: (402041043, level_crossing), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    2
geometry   count    2
Name: (402041043, pub), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    0
geometry   count    1
Name: (402041043, speed_camera), dtype: int64
osm_id     count    4
timestamp  count    0
name       count    0
geometry   count    4
Name: (402041043, traffic_signals), dtype: int64
osm_id     count    80
timestamp  count     0
name       count     0
geometry   count    80
Name: (402041043, turning_circle), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    0
geometry   count    1
Name: (402041044, bench), dtype: int64
osm_id     count    64
t

Name: (403011056, parking), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    1
geometry   count    2
Name: (403011056, pharmacy), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    0
geometry   count    3
Name: (403011056, post_box), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    2
geometry   count    2
Name: (403011056, post_office), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    1
geometry   count    1
Name: (403011056, pub), dtype: int64
osm_id     count    13
timestamp  count     0
name       count    12
geometry   count    13
Name: (403011056, restaurant), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    0
geometry   count    3
Name: (403011056, shelter), dtype: int64
osm_id     count    10
timestamp  count     0
name       count     0
geometry   count    10
Name: (403011056, shower), dtype: int64
osm_id     count    2
timestamp  count    0


Name: (403031068, parking), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    2
geometry   count    2
Name: (403031068, pharmacy), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    1
geometry   count    1
Name: (403031068, place_of_worship), dtype: int64
osm_id     count    7
timestamp  count    0
name       count    0
geometry   count    7
Name: (403031068, post_box), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    1
geometry   count    1
Name: (403031068, post_office), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    1
geometry   count    1
Name: (403031068, pub), dtype: int64
osm_id     count    4
timestamp  count    0
name       count    4
geometry   count    4
Name: (403031068, restaurant), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    1
geometry   count    2
Name: (403031068, shelter), dtype: int64
osm_id     count    1
timestamp  count    

Name: (404011096, clinic), dtype: int64
osm_id     count    43
timestamp  count     0
name       count     0
geometry   count    43
Name: (404011096, crossing), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    0
geometry   count    3
Name: (404011096, drinking_water), dtype: int64
osm_id     count    4
timestamp  count    0
name       count    4
geometry   count    4
Name: (404011096, fast_food), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    0
geometry   count    2
Name: (404011096, food_court), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    0
geometry   count    1
Name: (404011096, fountain), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    0
geometry   count    1
Name: (404011096, fuel), dtype: int64
osm_id     count    21
timestamp  count     0
name       count     0
geometry   count    21
Name: (404011096, information), dtype: int64
osm_id     count    1
timestamp  c

Name: (404021102, surveillance), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    0
geometry   count    1
Name: (404021102, telephone), dtype: int64
osm_id     count    7
timestamp  count    0
name       count    7
geometry   count    7
Name: (404021102, toilets), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    1
geometry   count    3
Name: (404021102, tower), dtype: int64
osm_id     count    43
timestamp  count     0
name       count     0
geometry   count    43
Name: (404021102, traffic_signals), dtype: int64
osm_id     count    20
timestamp  count     0
name       count     0
geometry   count    20
Name: (404021102, turning_circle), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    0
geometry   count    1
Name: (404021102, vending_machine), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    1
geometry   count    2
Name: (404021102, veterinary), dtype: int64
osm_id     count 

In [26]:
poi_df = pd.DataFrame()

In [27]:
poi_df["SA2_MAIN16"] = count.index.values
poi_df["poi_count"] = count[( 'geometry', 'count')].values

In [28]:
poi_df.head()

,SA2_MAIN16,poi_count
0,401011001,3288
1,401011002,756
2,401021004,279
3,401021010,153
4,401031011,276


In [29]:
split_count.shape

(1799, 4)

In [30]:
import math

In [31]:
entropy = {}
for i, row in split_count.iterrows():
    sa_id, _type = i
    total_count = poi_df.loc[poi_df["SA2_MAIN16"]==sa_id]["poi_count"]
    val = row[( 'geometry', 'count')]/total_count
    
    if sa_id not in entropy:
        entropy[sa_id] = (-val * np.log(val))
    else:
        entropy[sa_id] += (-val * np.log(val))

In [32]:
print(len(entropy))
print(len(poi_df))

76
76


In [33]:
entropy_list = []
for sa_id in poi_df.SA2_MAIN16:
    entropy_list.append(float(entropy[sa_id]))

In [34]:
poi_df["poi_count_entropy"] = entropy_list

In [35]:
poi_df.head()

,SA2_MAIN16,poi_count,poi_count_entropy
0,401011001,3288,2.487969
1,401011002,756,2.246419
2,401021004,279,2.616117
3,401021010,153,2.722838
4,401031011,276,2.472982


In [36]:
node_degree_entropy_df = node_degree_df.merge(poi_df,how="left",on="SA2_MAIN16")

In [37]:
node_degree_entropy_df.head()

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,...,sum_stay_duration_destination_entropy,total_stays_destination_entropy,num_nodes,num_1degree,num_2degree,num_3degree,num_4degree,num_greater5degree,poi_count,poi_count_entropy
0,401011001,41001,Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,3.949646,4.237396,1642,291,199,865,276,11,3288.0,2.487969
1,401011002,41002,North Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,3.152188,3.975034,517,149,54,239,69,6,756.0,2.246419
2,401021003,41003,Adelaide Hills,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.948900,2.042530,4034,1174,731,1982,141,6,NaN,NaN
3,401021004,41004,Aldgate - Stirling,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,1.099284,2.075796,2455,710,502,1126,110,7,279.0,2.616117
4,401021005,41005,Hahndorf - Echunga,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,1.654530,2.549629,989,286,245,421,36,1,NaN,NaN


In [38]:
node_degree_df.shape

(103, 48)

In [39]:
node_degree_entropy_df.shape

(103, 50)

In [40]:
node_degree_entropy_df.to_pickle('../../data_process/node_degree_entropy_df.pickle')

# Edge Degree DF

In [41]:
edge_degree_df.head()

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,...,num_2degree_x,num_3degree_x,num_4degree_x,num_greater5degree_x,num_nodes_y,num_1degree_y,num_2degree_y,num_3degree_y,num_4degree_y,num_greater5degree_y
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,...,199.0,865.0,276.0,11.0,1642.0,291.0,199.0,865.0,276.0,11.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,...,199.0,865.0,276.0,11.0,517.0,149.0,54.0,239.0,69.0,6.0
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003,838.0,82.0,...,199.0,865.0,276.0,11.0,4034.0,1174.0,731.0,1982.0,141.0,6.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,...,199.0,865.0,276.0,11.0,2455.0,710.0,502.0,1126.0,110.0,7.0
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005,517.0,13.0,...,199.0,865.0,276.0,11.0,989.0,286.0,245.0,421.0,36.0,1.0


In [42]:
entropy_x = []
notin = 0
for sa_id in edge_degree_df["sa2_code16_x"].values:
    if sa_id in entropy:
        entropy_x.append(float(entropy[sa_id]))
    else:
        notin += 1
print(notin)
notin=0
entropy_y = []
for sa_id in edge_degree_df["sa2_code16_y"]:
    if sa_id in entropy:
        entropy_y.append(float(entropy[sa_id]))
    else:
        notin += 1
print(notin)

3065
3122


In [43]:
notin

3122

In [44]:
edge_degree_df.shape

(10240, 39)

In [45]:
poi_df.head()

,SA2_MAIN16,poi_count,poi_count_entropy
0,401011001,3288,2.487969
1,401011002,756,2.246419
2,401021004,279,2.616117
3,401021010,153,2.722838
4,401031011,276,2.472982


In [46]:
sa_ids_poi = set(poi_df["SA2_MAIN16"].values)

In [47]:
edge_degree_df = edge_degree_df[edge_degree_df["sa2_code16_y"].isin(sa_ids_poi)]

In [48]:
print(edge_degree_df.shape)

(7118, 39)


In [49]:
edge_degree_df = edge_degree_df[edge_degree_df["sa2_code16_x"].isin(sa_ids_poi)]

In [50]:
print(edge_degree_df.shape)

(4983, 39)


In [51]:
count_dic = {key:val for key,val in zip(poi_df["SA2_MAIN16"].values, poi_df["poi_count"].values)}

In [52]:
entropy_x = []
poi_count_x = []
notin = 0
for sa_id in edge_degree_df["sa2_code16_x"].values:
    if sa_id in entropy:
        entropy_x.append(float(entropy[sa_id]))
        poi_count_x.append(float(count_dic[sa_id]))
    else:
        notin += 1
print(notin)
notin=0
entropy_y = []
poi_count_y = []
for sa_id in edge_degree_df["sa2_code16_y"]:
    if sa_id in entropy:
        entropy_y.append(float(entropy[sa_id]))
        poi_count_y.append(float(count_dic[sa_id]))
    else:
        notin += 1
print(notin)

0
0


In [53]:
print(len(entropy_x))
print(len(entropy_y))

4983
4983


In [54]:
edge_degree_df["poi_entropy_x"] = entropy_x
edge_degree_df["poi_entropy_y"] = entropy_y

In [55]:
edge_degree_df["poi_count_x"] = poi_count_x
edge_degree_df["poi_count_y"] = poi_count_y

In [56]:
edge_degree_df.shape

(4983, 43)

In [57]:
edge_degree_df.head()

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,...,num_nodes_y,num_1degree_y,num_2degree_y,num_3degree_y,num_4degree_y,num_greater5degree_y,poi_entropy_x,poi_entropy_y,poi_count_x,poi_count_y
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,...,1642.0,291.0,199.0,865.0,276.0,11.0,2.487969,2.487969,3288.0,3288.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,...,517.0,149.0,54.0,239.0,69.0,6.0,2.487969,2.246419,3288.0,756.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,...,2455.0,710.0,502.0,1126.0,110.0,7.0,2.487969,2.616117,3288.0,279.0
9,2018-02-01T00:00:00.000+11:00,401021010,4,401011001,685,347208,1897,401011001401021010,263.0,13.0,...,1854.0,535.0,360.0,911.0,46.0,2.0,2.487969,2.722838,3288.0,153.0
10,2018-02-01T00:00:00.000+11:00,401031011,4,401011001,2225,1664328,7709,401011001401031011,255.0,13.0,...,783.0,178.0,73.0,468.0,64.0,0.0,2.487969,2.472982,3288.0,276.0


In [58]:
edge_degree_df.to_pickle('../../data_process/edge_degree_entropy_df.pickle')